In [62]:
from difflib import SequenceMatcher
import regex as re
from tqdm import tqdm_notebook as tqdm
from Levenshtein import distance

In [63]:
txt_ref_path = "DATA/SOMUN.TN.txt"
txt_trg_path = "DATA/DYBG.TN.txt"

with open(txt_ref_path, 'r', encoding="utf-8") as fl:
    txt_ref_raw = fl.read()

with open(txt_trg_path, 'r', encoding="utf-8") as fl:
    txt_trg_raw = fl.read()

In [94]:
# hanzi 범위
cjk_range = "[\p{Han}]"
cjk_range_re = re.compile( cjk_range, re.UNICODE)

# hanzi 추출
txt_ref = "".join( re.findall( cjk_range_re, txt_ref_raw  ) )[10000:30000]
txt_trg = "".join( re.findall( cjk_range_re, txt_trg_raw  ) )[500000:510000]

txt_ref_len = len( txt_ref )
txt_trg_len = len( txt_trg )

print( txt_ref_len, txt_trg_len )

20000 10000


In [95]:
# params 

dist_cut_off = 3
unit_len = 8


In [96]:
rst = []

for i in tqdm( range( txt_ref_len - unit_len ) ):        # 소문
    unit_ref = txt_ref[i:i+unit_len]
    for j in range( txt_trg_len - unit_len ):    # 동의보감
        unit_trg = txt_trg[j:j+unit_len]
        dist = distance( unit_ref, unit_trg )
        if dist > dist_cut_off: continue
        rst.append( ( j-i, i, j )  )
        print(".", end="")

.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [97]:
ref_score = [0] * txt_ref_len
trg_score = [0] * txt_trg_len

In [98]:
for d, i, j in rst:
    for i_1 in range( i, i+unit_len ):
        ref_score[i_1] += 1
    for j_1 in range( j, j+unit_len ):
        trg_score[j_1] += 1

In [99]:
def non_zero_range( lst ):

    l = len( lst )
    idxs = []
    init = 0
    status = "off"
    
    for i, v in enumerate( lst ):
        if v != 0 :
            if status == "off":
                status = "on"
                init = i
            else:
                continue
        else:
            if status == "off":
                continue
            else:
                status = "off"
                idxs.append( (init, i) )
     
    return idxs


In [100]:
non_zero_range(ref_score)

[(7661, 7730),
 (10181, 10191),
 (12587, 12598),
 (13660, 13675),
 (13678, 13714),
 (14175, 14219),
 (16382, 16393),
 (16830, 16841),
 (16844, 16860),
 (16873, 16884),
 (16894, 16905),
 (16966, 16997),
 (17004, 17054)]

In [105]:
for b, e in non_zero_range(ref_score):
    print( txt_ref[b+2:e-2], end="\n\n" )


以夜行則喘出於腎淫氣病肺有所墮恐喘出於肝淫氣害脾有所驚恐喘出於肺淫氣傷心度水跌仆喘出於腎與骨當是之時勇者氣行則已怯者則着而爲病也故

病深者其聲噦

曰邪氣盛則實精

故犯賊風虛邪者陽受之食

者陰受之陽受之則入六府陰受之則入五藏入六府則身熱不時臥上爲喘呼入

曰陽明厥則喘而惋惋則惡人帝曰或喘而死者或喘而生者何也歧伯曰厥逆連藏則死連經則生帝

故不得臥臥則驚

臥而息有音者有

者有起居如故而息有音者有

不得臥臥而喘者

臥而息有音者是

夫起居如故而息有音者此肺之絡脈逆也絡脈不得隨經上下故留

故起居如故而息有音也夫不得臥臥則喘者是水氣之客也夫水者循津液而流也腎者水藏主津液主臥與喘也帝



In [106]:
for b, e in non_zero_range(trg_score):
    print( txt_trg[b+2:e-2], end="\n\n" )
    

理有起居如故而息有音者乃肺之絡脈逆而不得隨經上下故也

景夫不得臥臥則喘者是水氣之客也水者循津液而流也腎者水藏主津液主臥與喘也宜

急不得臥臥則喘者水氣

喘邪氣盛則實實

云犯賊風虛邪者陽受之陽受之則入六府入六府則身熱不時臥上爲喘呼又云陽明厥則喘而惋惋則惡人或喘而死者或喘而生者何也厥逆連藏則死連經則生此

人夜行則喘出於腎淫氣病肺有所墮恐喘出於肝淫氣害脾有所驚恐喘出於肺淫氣傷心渡水跌仆喘出於腎與骨當是之時勇者氣行則已怯者着而爲病也內

病甚者其聲噦



In [53]:
rst_sort = sorted( rst, key=lambda x: x[0] )



In [54]:
sorted( list( set( [ x[0] for x in rst ] ) ) )

[-85517,
 -81647,
 -76445,
 -76444,
 -76443,
 -75936,
 -75935,
 -75934,
 -35077,
 -35076,
 -35075,
 -26645,
 -26644,
 -26643,
 -26609,
 -26608,
 -26607,
 -26606,
 -26605,
 -26530,
 -26486,
 -26485,
 -26484,
 -26466,
 -23188,
 -23187,
 -23186,
 -23185,
 -23045,
 -23044,
 -23043,
 -22554]

In [57]:
last = 0
rst2 = []
local_i = []
local_j = []
  
for delta, i, j in rst_sort:
    if last == 0:
        last = delta 
    if last != delta: 
        rst2.append( ( ( local_i[0], local_i[-1] + unit_len ), ( local_j[0], local_j[-1] + unit_len) ) )
        last = delta
        local_i = []
        local_j = []
    local_i.append( i )
    local_j.append( j )

In [58]:
for (i_b,i_e), (j_b, j_e) in rst2:
    print( "R: {}\nT: {}\n\n".format( txt_ref[i_b:i_e], txt_trg[j_b:j_e ] ) )

R: 脾氣虛則夢飮食不足
T: 治氣虛喘急飮食不進


R: 寃者是腎氣之逆也一
T: 喘者是水氣之客也水


R: 大滿膨膨而喘咳病本
T: 肺脹滿膨膨而喘咳所


R: 䐜腹大滿膨膨而喘咳病本于
T: 則肺脹滿膨膨而喘咳所生病


R: 腹大滿膨膨而喘咳病
T: 脹滿膨膨而喘咳所生


R: 大滿膨膨而喘咳病本
T: 肺脹滿膨膨而喘咳者


R: 䐜腹大滿膨膨而喘咳病本于
T: 景肺脹滿膨膨而喘咳者多用


R: 腹大滿膨膨而喘咳病
T: 脹滿膨膨而喘咳者多


R: 者陰氣在下陽氣在上諸陽
T: 散治陰氣在下陽氣在上咳


R: 氣喘者陰氣在下陽氣在上諸陽氣
T: 白散治陰氣在下陽氣在上咳嘔喘


R: 喘者陰氣在下陽氣在上諸
T: 治陰氣在下陽氣在上咳嘔


R: 故起居如故而息有音也夫
T: 理有起居如故而息有音者


R: 也微故起居如故而息有音也夫不
T: 明理有起居如故而息有音者乃肺


R: 微故起居如故而息有音也
T: 有起居如故而息有音者乃


R: 脈不得隨經上下故留經
T: 逆而不得隨經上下故也


R: 也絡脈不得隨經上下故留經而
T: 脈逆而不得隨經上下故也入門


R: 夫起居如故而息有音者此肺之絡脈逆也絡脈不得隨經上下故留
T: 理有起居如故而息有音者乃肺之絡脈逆而不得隨經上下故也入


R: 謂也夫起居如故而息有音者此肺之絡脈逆也絡脈
T: 明理有起居如故而息有音者乃肺之絡脈逆而不得


R: 也夫起居如故而息有音者此肺之絡脈逆
T: 有起居如故而息有音者乃肺之絡脈逆而


R: 不得臥而息有音者是陽明
T: 居如故而息有音者乃肺之


R: 者有起居如故而息有音者有得
T: 明理有起居如故而息有音者乃


R: 無音者有起居如故而息有音者有得臥
T: 也明理有起居如故而息有音者乃肺之


R: 音者有起居如故而息有音者有
T: 理有起居如故而息有音者乃肺


R: 不得臥而息有音者有不得
T: 居如故而息有音者乃肺之


R: 夫水者循津液而流也腎者水藏主津液主臥與喘也帝曰
T: 客也水者循津液而流也腎者水藏主津液主臥與喘也宜


R: 也夫不得臥臥則喘者是水氣之客也夫水者循津液而流也腎者水藏主津液主臥與喘也帝曰善
T: 仲景夫不得臥臥則喘者是水氣之客也水者循津液而流也腎者水藏主津液主臥與喘也宜神秘


R: 有音也夫不得臥臥則

In [59]:
rst

[(-22554, 26382, 3828),
 (-22554, 26383, 3829),
 (-22554, 26384, 3830),
 (-22554, 26385, 3831),
 (-26466, 26830, 364),
 (-26466, 26831, 365),
 (-26466, 26832, 366),
 (-26466, 26833, 367),
 (-26485, 26844, 359),
 (-26485, 26845, 360),
 (-26484, 26845, 361),
 (-26486, 26846, 360),
 (-26485, 26846, 361),
 (-26484, 26846, 362),
 (-26486, 26847, 361),
 (-26485, 26847, 362),
 (-26484, 26847, 363),
 (-26486, 26848, 362),
 (-26485, 26848, 363),
 (-26484, 26848, 364),
 (-26486, 26849, 363),
 (-26485, 26849, 364),
 (-26484, 26849, 365),
 (-26486, 26850, 364),
 (-26485, 26850, 365),
 (-26484, 26850, 366),
 (-26486, 26851, 365),
 (-26485, 26851, 366),
 (-26485, 26852, 367),
 (-23044, 26873, 3829),
 (-23044, 26874, 3830),
 (-23043, 26874, 3831),
 (-23045, 26875, 3830),
 (-23044, 26875, 3831),
 (-23044, 26876, 3832),
 (-26530, 26894, 364),
 (-26530, 26895, 365),
 (-26530, 26896, 366),
 (-26530, 26897, 367),
 (-26606, 26966, 360),
 (-26606, 26967, 361),
 (-26605, 26967, 362),
 (-26607, 26968, 361),
 

In [40]:
print( txt_ref[47713:47717+8] )
print( txt_trg[3772:3776+8] )

日一痏二日二痏漸多之十五
下一粒二日二粒漸加至十五
